In [2]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import sys
import numpy as np

%tensorflow_version 2.x
import tensorflow as tf
import os
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from astropy.nddata.utils import Cutout2D
import matplotlib.cm as cm
import matplotlib as mpl
import astropy.units as u
from collections import Counter
from matplotlib.ticker import FuncFormatter
import matplotlib.ticker as ticker
print(tf.__version__)

from google.colab import drive
drive.mount('/content/drive')

!nvidia-smi

class GAN():
    def __init__(self):

        self.img_rows = 100
        self.img_cols = 100
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build and compile the generator
        self.generator = self.build_generator()
        self.generator.compile(loss='binary_crossentropy', optimizer=optimizer)

        # The generator takes noise as input and generated imgs
        z = Input(shape=(500,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The valid takes generated images as input and determines validity
        valid = self.discriminator(img)

        # The combined model  (stacked generator and discriminator) takes
        # noise as input => generates images => determines validity
        self.combined = Model(z, valid)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    def build_generator(self):

        noise_shape = (500,)

        model = Sequential()

        model.add(Dense(256, input_shape=noise_shape))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        model.add(Reshape(self.img_shape))

        model.summary()

        noise = Input(shape=noise_shape)
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        img_shape = (self.img_rows, self.img_cols, self.channels)

        model = Sequential()

        model.add(Flatten(input_shape=img_shape))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(524))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        img = Input(shape=img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, save_interval=50):

        self.ptm = ''
        self.filename = '154000'
        self.nx = 480
        self.ny = 480
        self.nz = 256
        
        ################################
        # Charging the data into the code - every data is converted into a cube 
        # of data so that it has the form of the dominium of the simulation
        #
        # The lines of the code for mvxx, mvyy, mbxx and mbyy are commented beacuse
        # we are not interested in ploting this magnitudes for the image analysis 
        # we are going to make
        #
        # The temperature is obtained from the data file related to the 
        # equation of state (EOS)
        ################################
  
        
        print("reading EOS")
        self.mtpr = np.fromfile(self.ptm+"eos."+self.filename,dtype=np.float32)
        self.mtpr = np.reshape(self.mtpr, (2, self.nx,self.nz, self.ny), order="A")
        n_eos = 0
        self.mtpr = self.mtpr[n_eos,:,:,:] 
        # n_eos -> 0: temperature ; 1: pressure
        print("EOS done")


        plot_mtpr = np.zeros((self.nx, self.ny))
        
        nz_slice = 179
        
#       for i in range(np.shape(self.mbzz)[0]):

        plot_mtpr = self.mtpr[380:,nz_slice,380:]
        print(np.shape(plot_mtpr))

        # Load the dataset
        X_train = plot_mtpr

        # Rescale -1 to 1
        X_train = X_train.reshape(1, 100, 100, 1).astype('float32')
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        #X_train = np.expand_dims(X_train, axis=1)

        half_batch = int(batch_size / 2)

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random half batch of images
            idx = np.random.randint(0, X_train.shape[0], half_batch)
            imgs = X_train[idx]

            noise = np.random.normal(0, 1, (half_batch, 500))

            # Generate a half batch of new images
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(imgs, np.ones((half_batch, 1)))
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)


            # ---------------------
            #  Train Generator
            # ---------------------

            noise = np.random.normal(0, 1, (batch_size, 500))

            # The generator wants the discriminator to label the generated samples
            # as valid (ones)
            valid_y = np.array([1] * batch_size)

            # Train the generator
            g_loss = self.combined.train_on_batch(noise, valid_y)

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % save_interval == 0:
                self.save_imgs(epoch)

    def save_imgs(self, epoch):
        r, c = 1, 1
        noise = np.random.normal(0, 1, (r * c, 500))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                plt.imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                cnt += 1
        fig.savefig("/mnist_%d.png" % epoch)
        plt.close()


if __name__ == '__main__':
    gan = GAN()
    gan.train(epochs=31000, batch_size=32, save_interval=200)

Se han truncado las últimas 5000 líneas del flujo de salida.
26000 [D loss: 0.000077, acc.: 100.00%] [G loss: 8.779500]
26001 [D loss: 0.000092, acc.: 100.00%] [G loss: 8.760157]
26002 [D loss: 0.000090, acc.: 100.00%] [G loss: 8.756760]
26003 [D loss: 0.000091, acc.: 100.00%] [G loss: 8.805105]
26004 [D loss: 0.000099, acc.: 100.00%] [G loss: 8.735420]
26005 [D loss: 0.000094, acc.: 100.00%] [G loss: 8.811422]
26006 [D loss: 0.000082, acc.: 100.00%] [G loss: 8.770310]
26007 [D loss: 0.000080, acc.: 100.00%] [G loss: 8.769582]
26008 [D loss: 0.000091, acc.: 100.00%] [G loss: 8.739858]
26009 [D loss: 0.000088, acc.: 100.00%] [G loss: 8.803534]
26010 [D loss: 0.000093, acc.: 100.00%] [G loss: 8.766733]
26011 [D loss: 0.000096, acc.: 100.00%] [G loss: 8.702768]
26012 [D loss: 0.000067, acc.: 100.00%] [G loss: 8.783992]
26013 [D loss: 0.000102, acc.: 100.00%] [G loss: 8.774113]
26014 [D loss: 0.000075, acc.: 100.00%] [G loss: 8.840191]
26015 [D loss: 0.000089, acc.: 100.00%] [G loss: 8.803